In [85]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

In [86]:
path = Path(os.path.abspath(''))

In [87]:
path

WindowsPath('C:/Users/bartlomiej/Documents/Studia/Music-instrument-recognition')

In [109]:
#Spectrogram
train_x = np.load(str(path) + '\\train_X.npy')
train_y = np.load(str(path) + '\\train_y.npy')

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [111]:
print(train_x.shape)
print(test_x.shape)

(4693, 128, 130)
(2012, 128, 130)


In [113]:
train_x = train_x.reshape(4693,128,130,1)
test_x = test_x.reshape(2012,128,130,1)

In [114]:
from keras.layers import Input
trainInput = Input(shape=(128,130,1))

x = Conv2D(128, kernel_size=3, activation='relu')(trainInput)
x = Conv2D(64, kernel_size=3, activation='relu')(x)
x = Conv2D(32, kernel_size=3, activation='relu')(x)
x = Flatten()(x)
x = Dense(11, activation='sigmoid')(x)

In [115]:
from keras.models import Model
xModel = Model(inputs=trainInput, outputs=x)

[<tf.Tensor 'input_22:0' shape=(None, 128, 130, 1) dtype=float32>]


In [116]:
#Parameters
from sklearn.model_selection import train_test_split
import cv2
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

data = pd.read_csv('C:/Users/bartlomiej/Documents/Studia/Music-instrument-recognition/Sieci/data.csv')
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)


In [117]:
print(x_train.shape)
print(x_test.shape)

(4693, 25)
(2012, 25)


In [118]:
trainInput2 = Input(shape=(25,))

y = Dense(128, activation='relu')(trainInput2)
y = Dense(64, activation='relu')(y)
y = Dense(32, activation='relu')(y)
y = Dense(11, activation='sigmoid')(y)

In [119]:
yModel = Model(inputs=trainInput2, outputs=y)

In [120]:
from keras.layers import concatenate
combined = concatenate([xModel.output, yModel.output])

In [121]:
z = Dense(4, activation="relu")(combined)
z = Dense(1, activation="linear")(z)

In [123]:
model = Model(inputs=[xModel.input, yModel.input], outputs=x)
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [126]:
fit = model.fit([train_x, x_train], train_y, validation_data=([test_x, x_test], test_y), epochs=3)

Train on 4693 samples, validate on 2012 samples
Epoch 1/3
4693/4693 [==============================] - 482s 103ms/step - loss: 0.3016 - accuracy: 0.8983 - val_loss: 0.3021 - val_accuracy: 0.8941
Epoch 2/3
4693/4693 [==============================] - 473s 101ms/step - loss: 0.2163 - accuracy: 0.9201 - val_loss: 0.3732 - val_accuracy: 0.8935
Epoch 3/3
4693/4693 [==============================] - 479s 102ms/step - loss: 0.1268 - accuracy: 0.9538 - val_loss: 0.5343 - val_accuracy: 0.8837


In [127]:
model.reset_metrics()

In [128]:
model.save('ir-both-model-v_0_3.h5')

In [129]:
print(model.input)

[<tf.Tensor 'input_22:0' shape=(None, 128, 130, 1) dtype=float32>, <tf.Tensor 'input_23:0' shape=(None, 25) dtype=float32>]
